In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
pip install feedparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.9 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=1d33522229a78a07b582cf189e319b1f13d300d7efd345b452a045afb2375a16
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [4]:
import feedparser
import csv
import pandas as pd
import re

In [5]:
our_feeds = {'Kommersant': 'https://www.kommersant.ru/RSS/section-economics.xml', 
'KommersantBus': 'https://www.kommersant.ru/RSS/section-business.xml', 
'Lenta.ru': 'https://lenta.ru/rss/news/economics', 
'Garant': 'http://rss.garant.ru/categories/news/', 
'Nalogi' : 'http://rss.garant.ru/consult/nalog/', 
'MK' : 'https://www.mk.ru/rss/economics/index.xml', 
'News' : 'https://news.ru/rss/category/post/economics/'} #пример словаря RSS-лент 
                                                                  #русскоязычных источников

f_all_news = 'allnews23march.csv' 
f_certain_news = 'certainnews23march.csv' #пример пути файла

vector1 = 'ДолЛАР|РубЛ|ЕвРО' #пример таргетов
vector2 = 'ЦБ|СбЕРбАНК|курс'

In [6]:
def check_url(url_feed): #функция получает линк на рсс ленту, возвращает        
# распаршенную ленту с помощью feedpaeser
    return feedparser.parse(url_feed)  
    
def getHeadlines(url_feed): #функция для получения заголовков новости
    headlines = []
    lenta = check_url (url_feed)
    for item_of_news in lenta['items']:
        headlines.append(item_of_news ['title'])
    return headlines

def getDescriptions(url_feed): #функция для получения описания новости
    descriptions = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        descriptions.append(item_of_news ['description'])
    return descriptions

def getLinks(url_feed): #функция для получения ссылки на источник новости
    links = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        links.append(item_of_news ['link'])
    return links

def getDates(url_feed): #функция для получения даты публикации новости
    dates = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        dates.append(item_of_news ['published'])
    return dates

In [7]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем наши URL и добавляем их в наши пустые списки
for key,url in our_feeds.items():
    allheadlines.extend( getHeadlines(url) )
    
for key,url in our_feeds.items():
    alldescriptions.extend( getDescriptions(url) )
    
for key,url in our_feeds.items():
    alllinks.extend( getLinks(url) )
    
for key,url in our_feeds.items():
    alldates.extend( getDates(url) )

In [8]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, 
# возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
        
            
    return df

In [9]:
def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                #определенных новостей по таргета,
                #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [10]:
df = write_all_news(f_all_news) #все новости
df.to_csv(r'/content/export_dataframe.csv', index = False, header=True, encoding='utf-8')

In [11]:
write_all_news(f_all_news)

,Title,Description,Links,Publication Date
0,Предприниматели купили 500 тыс. лицензий на от...,Программа Минцифры позволяет купить компаниям ...,https://www.klerk.ru/buh/news/537139/,"Sat, 08 Oct 2022 14:16:00 +0300"
1,Скоро в России появятся мобильные грузоматы. И...,Первыми клиентами станут строительные и промыш...,https://www.klerk.ru/buh/news/537134/,"Sat, 08 Oct 2022 13:37:00 +0300"
2,🚩 Получить аккредитацию IT-компания может с уч...,Для первого варианта аккредитации кроме нужног...,https://www.klerk.ru/buh/news/537154/,"Sat, 08 Oct 2022 12:14:00 +0300"
3,Работодатель не платит мобилизованному зарплат...,"Мобилизованных увольнять нельзя, однако зарабо...",https://www.klerk.ru/buh/news/537159/,"Sat, 08 Oct 2022 11:24:00 +0300"
4,С 1 марта 2023 года перемещаться на электросам...,"Элеткросамокатам, электроскейтбордам, сигвеям,...",https://www.klerk.ru/buh/news/537132/,"Sat, 08 Oct 2022 10:00:00 +0300"
...,...,...,...,...
146,Участие России в МВФ предложили пересмотреть и...,«Американцы закрывают потенциальную лазейку»,https://www.mk.ru/economics/2022/10/05/uchasti...,"Wed, 05 Oct 2022 13:51:42 +0300"
147,Страны ЕС согласовали введение потолка цен на ...,"Страны, входящие в Евросоюз, согласовали введе...",https://www.mk.ru/economics/2022/10/05/strany-...,"Wed, 05 Oct 2022 12:01:57 +0300"
148,Всемирный банк сравнил падение экономик России...,Украинская экономика сократится в восемь раз б...,https://www.mk.ru/economics/2022/10/05/vsemirn...,"Wed, 05 Oct 2022 09:45:10 +0300"
149,В США нашли виновного в атаках на «Северные по...,Исследователь Эрик Цуэссе в интервью телеканал...,https://www.mk.ru/economics/2022/10/05/v-ssha-...,"Wed, 05 Oct 2022 07:31:10 +0300"


In [12]:
from google.colab import files 

In [13]:
files.download('/content/export_dataframe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [76]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) 
    

,Title,Description,Links,Publication Date
29,Курс доллара. Прогноз на 10-14 октября // Мнен...,Минувшую неделю рубль завершил сильным ослабле...,https://www.kommersant.ru/doc/5605709,"Fri, 07 Oct 2022 21:39:00 +0300"
33,Индекс РТС опустился ниже 1000 пунктов впервые...,Долларовый индекс РТС на Мосбирже снижался до ...,https://www.kommersant.ru/doc/5605644,"Fri, 07 Oct 2022 18:01:00 +0300"
51,Доллар подорожал до 61 рубля,После открытия торгов на Мосбирже доллар США п...,https://www.kommersant.ru/doc/5596123,"Fri, 07 Oct 2022 08:36:00 +0300"
76,Евросоюз захлопнул криптокошельки // Российски...,Новый пакет санкций ЕС в значительной мере зат...,https://www.kommersant.ru/doc/5595843,"Thu, 06 Oct 2022 19:18:00 +0300"
93,Объем частных инвестиций в российские акции в ...,В сентябре 2022 года частные инвесторы вложили...,https://www.kommersant.ru/doc/5595593,"Thu, 06 Oct 2022 11:50:00 +0300"
127,От европейских банков потребовали экономии,Европейский центробанк (ЕЦБ) ужесточает требов...,https://lenta.ru/news/2022/10/07/ecbbanki/,"Fri, 07 Oct 2022 17:00:00 +0300"
130,Курс евро превысил 61 рубль впервые с 21 сентября,Курс евро в ходе торгов на Московской бирже 7 ...,https://lenta.ru/news/2022/10/07/euro/,"Fri, 07 Oct 2022 15:28:03 +0300"
134,Курс Binance Coin резко упал после кражи милли...,В ходе торгов 7 октября курс криптовалюты Bina...,https://lenta.ru/news/2022/10/07/bnb/,"Fri, 07 Oct 2022 14:56:00 +0300"
224,Курс доллара превысил 60 рублей,Курс доллара в ходе торгов на Московской бирже...,https://lenta.ru/news/2022/10/05/doll/,"Wed, 05 Oct 2022 16:06:00 +0300"
342,Месяц валютного перелома: эксперты дали прогно...,Курс доллара в ходе торгов 6 октября превысил ...,https://www.mk.ru/economics/2022/10/06/mesyac-...,"Thu, 06 Oct 2022 21:04:16 +0300"


AttributeError: ignored